In [66]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in
import warnings
warnings.filterwarnings("ignore")

import numpy as np
from sklearn.base import BaseEstimator,TransformerMixin, ClassifierMixin
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.linear_model import ElasticNetCV, LassoLarsCV, LinearRegression, HuberRegressor, RANSACRegressor
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.pipeline import make_pipeline, make_union
from sklearn.utils import check_array
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.random_projection import GaussianRandomProjection
from sklearn.random_projection import SparseRandomProjection
from sklearn.decomposition import PCA, FastICA
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor

In [5]:
class StackingEstimator(BaseEstimator, TransformerMixin):

    def __init__(self, estimator):
        self.estimator = estimator

    def fit(self, X, y=None, **fit_params):
        self.estimator.fit(X, y, **fit_params)
        return self
    def transform(self, X):
        X = check_array(X)
        X_transformed = np.copy(X)
        # add class probabilities as a synthetic feature
        if issubclass(self.estimator.__class__, ClassifierMixin) and hasattr(self.estimator, 'predict_proba'):
            X_transformed = np.hstack((self.estimator.predict_proba(X), X))

        # add class prodiction as a synthetic feature
        X_transformed = np.hstack((np.reshape(self.estimator.predict(X), (-1, 1)), X_transformed))

        return X_transformed



In [6]:
import pickle
# train = pickle.load(open('train.pkl', 'rb'))
train = pd.read_csv('../../data/train2.csv')
original_train = pd.read_csv('../../data/train2.csv')
usable_columns = original_train.drop('y', axis=1).columns
test = pd.read_csv('../../data/test.csv')
public_lb = pd.read_csv('../../data/public_lb.csv')
public_lb['ID'] = public_lb.id
public_lb['y'] = public_lb.yValue

best = pd.read_csv('v6_probing.csv')

In [59]:
for c in train.columns:
    if train[c].dtype == 'object':
        lbl = LabelEncoder()
        lbl.fit(list(train[c].values) + list(test[c].values))
        train[c] = lbl.transform(list(train[c].values))
        test[c] = lbl.transform(list(test[c].values))



n_comp = 12

# tSVD
tsvd = TruncatedSVD(n_components=n_comp, random_state=420)
tsvd_results_train = tsvd.fit_transform(train.drop(["y"], axis=1))
tsvd_results_test = tsvd.transform(test)

# PCA
pca = PCA(n_components=n_comp, random_state=420)
pca2_results_train = pca.fit_transform(train.drop(["y"], axis=1))
pca2_results_test = pca.transform(test)

# ICA
ica = FastICA(n_components=n_comp, random_state=420)
ica2_results_train = ica.fit_transform(train.drop(["y"], axis=1))
ica2_results_test = ica.transform(test)

# GRP
grp = GaussianRandomProjection(n_components=n_comp, eps=0.1, random_state=420)
grp_results_train = grp.fit_transform(train.drop(["y"], axis=1))
grp_results_test = grp.transform(test)

# SRP
srp = SparseRandomProjection(n_components=n_comp, dense_output=True, random_state=420)
srp_results_train = srp.fit_transform(train.drop(["y"], axis=1))
srp_results_test = srp.transform(test)

#save columns list before adding the decomposition components


# Append decomposition components to datasets
for i in range(1, n_comp + 1):
    train['pca_' + str(i)] = pca2_results_train[:, i - 1]
    test['pca_' + str(i)] = pca2_results_test[:, i - 1]

    train['ica_' + str(i)] = ica2_results_train[:, i - 1]
    test['ica_' + str(i)] = ica2_results_test[:, i - 1]

    train['tsvd_' + str(i)] = tsvd_results_train[:, i - 1]
    test['tsvd_' + str(i)] = tsvd_results_test[:, i - 1]

    train['grp_' + str(i)] = grp_results_train[:, i - 1]
    test['grp_' + str(i)] = grp_results_test[:, i - 1]

    train['srp_' + str(i)] = srp_results_train[:, i - 1]
    test['srp_' + str(i)] = srp_results_test[:, i - 1]

y_train = train['y'].values
y_mean = np.mean(y_train)
id_test = test['ID'].values
#finaltrainset and finaltestset are data to be used only the stacked model (does not contain PCA, SVD... arrays)
# usable_columns=['ID', 'X0', 'X47','X95','X314','X315','X232','X119','X311','X76','X329','X238','X340','X362','X137']
usable_columns= ['ID', 'X118',
            'X127',
            'X47',
            'X315',
            'X311',
            'X179',
            'X314',
### added by Tilii
            'X232',
            'X29',
            'X263',
###
            'X261']
finaltrainset = train[usable_columns]
finaltestset = test[usable_columns]

In [67]:
stacked_pipeline = make_pipeline(
    StackingEstimator(estimator=LassoLarsCV(normalize=True)),
    StackingEstimator(estimator=GradientBoostingRegressor(learning_rate=0.001, loss="huber", max_depth=3, max_features=0.55, min_samples_leaf=18, min_samples_split=14, subsample=0.7)),
    LassoLarsCV()
)
final1 = KNeighborsRegressor(n_neighbors=80,weights='uniform',p=2)
final2 = KNeighborsRegressor(n_neighbors=80,weights='uniform',p=2)
final3 = KNeighborsRegressor(n_neighbors=80,weights='uniform',p=2)
final0 = KNeighborsRegressor(n_neighbors=80,weights='uniform',p=2)

In [68]:
from sklearn.cluster import KMeans

x_train = train[usable_columns]
clstr = KMeans(n_clusters=4)
clstr.fit(finaltrainset)
res = clstr.predict(finaltrainset)



In [69]:
final0.fit(finaltrainset[res == 0], y_train[res == 0])
final1.fit(finaltrainset[res == 1], y_train[res == 1])
final2.fit(finaltrainset[res == 2], y_train[res == 2])
final3.fit(finaltrainset[res == 3], y_train[res == 3])

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=1, n_neighbors=80, p=2,
          weights='uniform')

In [70]:
rest_test = clstr.predict(finaltestset[usable_columns])
predict0 = final0.predict(finaltestset[rest_test == 0])
predict1 = final1.predict(finaltestset[rest_test == 1])
predict2 = final2.predict(finaltestset[rest_test == 2])
predict3 = final3.predict(finaltestset[rest_test == 3])

In [71]:
f0 = finaltestset[rest_test == 0]
f0['y'] = predict0
f1 = finaltestset[rest_test == 1]
f1['y'] = predict1
f2 = finaltestset[rest_test == 2]
f2['y'] = predict2
f3 = finaltestset[rest_test == 3]
f3['y'] = predict3

final_test = f0.append(f1, ignore_index=True)
final_test = final_test.append(f2, ignore_index=True)
final_test = final_test.append(f3, ignore_index=True)
final_test = final_test.sort_values('ID', ascending=True)
stack_predsTM = final_test.y

In [72]:
# y_pred = model.predict(dtest)
# results = stacked_pipeline.predict(finaltestset)

sub = pd.DataFrame()
sub['ID'] = id_test
sub['y'] = stack_predsTM#y_pred*0.75 + results*0.25
res = sub[sub.ID.isin(public_lb.id)]
result = r2_score(public_lb.yValue, res.y)
print("r2 to probe : ", r2_score(public_lb.yValue, res.y))
print("mse  to probe : ", mean_squared_error(public_lb.yValue, res.y))
print("mae  to probe: ", mean_absolute_error(public_lb.yValue, res.y))
exception = False
# res.shape

r2 to probe :  -0.0576536994867
mse  to probe :  210.340310754
mae  to probe:  12.092617532


Base Results = 0.720429268928

In [51]:
first = sub[sub.ID.isin(public_lb.id) == False]
second = public_lb[['ID', 'y']]
sub2 = first.append(second, ignore_index=True)
sub2 = sub2.sort_values('ID', ascending=True)

In [52]:
print(r2_score(best.y, sub2.y ))

0.982318574813


In [53]:
#sanity check
r2_score(public_lb.yValue, sub2[sub2.ID.isin(public_lb.id)].y)

1.0

In [55]:
sanity_check= sub2[pd.isnull(sub2.y)].shape[0]
print(sub2.shape)
print(sanity_check == 0)
print(sub2.shape[0] == test.shape[0])
if sanity_check == 0 and sub2.shape[0] == test.shape[0] and (result + 0.001 > 0.720429268928 or exception):
    sub2.to_csv('stacked-models7.csv', index=False)
    print("saved")
else:
    print("below can't continue")

(4209, 2)
True
True
saved
